## Att leka med kluster av mat

Det finns över tvåtusen livsmedel i Livsmedelsdatabasen, och de har värden för över 50 näringsvärden.

Att sortera dem i Excel funkar ett tag, men det blir svårt att få en samlad bedömning eller gruppering av stora mängder livsmedel.

Det finns också en metadataklassificering som Livsmedelsverket kallar "huvudgrupp". Det är en klassificering som bara har en nivå. Inget över, inget under. En flat nivå.

In [1]:
import sqlite3
import numpy as np
from sklearn.cluster import KMeans

In [2]:
debug = True

Börja med att läsa in allt från databasen. Det tar inte så mycket minne, men är svårt att få överblick över.

Allt ligger i variabeln dataset som är en numpy-array. Man kan skriva ut den eller delar av den när som helst om man vill se vad som ligger på en viss rad eller en viss kolumn. Kolumn 0 är livsmedelsnamnet, 1 är livsmedelsnumret.

In [3]:
conn = sqlite3.connect('livs.db')
conn.row_factory = sqlite3.Row
curs = conn.cursor()

result = []
for row in curs.execute('select * from livs'):
    result.append(row)

conn.close()

dataset = np.array(result)
if debug:
    print (dataset[:5,:5])

[['Talg nöt' '1' 656.3 2746.0 0.0]
 ['Späck gris' '2' 763.0 3192.6 0.0]
 ['Ister gris' '3' 884.3 3700.0 0.0]
 ['Kokosfett' '4' 884.3 3700.0 0.0]
 ['Matfettsblandning havssaltat fett 80% berikad typ Bregott' '5' 711.5
  2977.0 0.5]]


Livsmedelsverkets gruppering har jag lagt i kolumn 60. Numpy-funktionen np.unique() returnerar både alla unika värden och (om man vill) hur många som är i varje kategori.

In [4]:
unique, counts = np.unique(dataset.T[60], return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
huvudgrupp_storlek = np.array(list(zip(unique, counts)))

Skriver man ut huvudgrupp_storlek blir det typ 118 kategorier i bokstavsordning. Här är de n första.

In [5]:
n = 20
print (huvudgrupp_storlek[:n])

[['' '7']
 ['Algprodukter' '1']
 ['Baljväxter (bönor, linser och ärter)' '51']
 ['Blodmat' '2']
 ['Blodprodukter o rätter' '8']
 ['Buljong' '18']
 ['Bullar kakor tårtor mm' '86']
 ['Bär färska frysta' '24']
 ['Chips popcorn o dyl' '11']
 ['Choklad' '7']
 ['Cider alkoläsk drink' '3']
 ['Deg och gräddade skal och bottnar' '3']
 ['Dessertost' '8']
 ['Dryck' '1']
 ['Efterrätter' '31']
 ['Fisk färsk fryst kokt' '68']
 ['Fisk o skaldjursprodukter o rätter' '69']
 ['Fisk rökt' '13']
 ['Fisk stekt ej panerad' '7']
 ['Flingor - frukostflingor' '28']]


Jag förstår inte riktigt varför det är så svårt att sortera den arrayen. Det är ju en sak att allting är strängar, eftersom en numpy-array bara kan ha en enda datatyp, men det borde finnas ett enkelt sätt att sortera efter kolumn två som integer...

Som det blir nu behöver man skapa ett index genom att använda np.argsort, ta de 20 första med [-20:] och sen vända på ordningen med [::-1], som helt enkelt är en slice till, där -1 är step-argumentet. https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html 

In [6]:
antal = 20
index = np.argsort(huvudgrupp_storlek.T[1].astype(int))[-antal:][::-1] #https://stackoverflow.com/questions/6910641/how-to-get-indices-of-n-maximum-values-in-a-numpy-array
print(huvudgrupp_storlek[index])

[['Kött färskt fryst tillagat' '127']
 ['Grönsaker' '94']
 ['Bullar kakor tårtor mm' '86']
 ['Köttprodukter kötträtter' '80']
 ['Sås dressing majonnäs' '79']
 ['Fisk o skaldjursprodukter o rätter' '69']
 ['Fisk färsk fryst kokt' '68']
 ['Korv' '60']
 ['Grönsaks- rotfrukts- baljväxträtter och produkter' '55']
 ['Baljväxter (bönor, linser och ärter)' '51']
 ['Soppa mat' '47']
 ['Pizza paj pirog färdig smörgås' '46']
 ['Frukt färsk fryst' '42']
 ['Mjukt bröd' '42']
 ['Hård matfettsblandning' '39']
 ['Fågel' '39']
 ['Potatisprodukter o rätter' '33']
 ['Inälvor och organ' '33']
 ['Potatis' '31']
 ['Efterrätter' '31']]


Medan de största grupperna är ganska rättframma är de minsta desto knepigare. Vad ska man göra med en klassificering som innehåller en huvudgrupp "Risrätter" med endast ett livsmedel, och det är paella och inte exempelvis risgrynsgröt?

In [7]:
index = np.argsort(huvudgrupp_storlek.T[1].astype(int))[:20] #https://stackoverflow.com/questions/6910641/how-to-get-indices-of-n-maximum-values-in-a-numpy-array
print(huvudgrupp_storlek[index])

[['Sötningsmedel' '1']
 ['Algprodukter' '1']
 ['Kryddor' '1']
 [ 'Övrigt animaliskt *kött*, grodlår, sniglar, säl - färskt, fryst, tillagat'
  '1']
 ['Sockerfritt godis' '1']
 ['Risrätter' '1']
 ['Dryck' '1']
 ['Tacoskal' '1']
 ['Riskakor' '2']
 ['Frukt o nötblandningar bars' '2']
 ['Frukt o bär' '2']
 ['Sportdrycker energidrycker' '2']
 ['Gelatin agar agar' '2']
 ['Övrigt' '2']
 ['Blodmat' '2']
 ['Tuggummi' '2']
 ['Kakaoprodukter' '2']
 ['Cider alkoläsk drink' '3']
 ['Osträtter' '3']
 ['Smör' '3']]


Det går ju även att få fram vilka livsmedel som finns i de där grupperna.

In [8]:
for row in huvudgrupp_storlek[index]:
    q = np.where(dataset.T[60]==row[0])
    print(row[0],"\n",dataset[q,0]) #Det går ju att skriva ut vilka värden som helst för varje livsmedel, som med dataset[q,:]

Sötningsmedel 
 [['Sorbitol m sackarin']]
Algprodukter 
 [['Kelp torkad']]
Kryddor 
 [['Kryddblandning taco']]
Övrigt animaliskt *kött*, grodlår, sniglar, säl - färskt, fryst, tillagat 
 [['Grodlår råa frysta']]
Sockerfritt godis 
 [['Karameller syrliga sockerfria']]
Risrätter 
 [['Paella']]
Dryck 
 [['Mandeldryck']]
Tacoskal 
 [['Tacoskal']]
Riskakor 
 [['Riskakor fullkorn solrosfrön majs fett 4%'
  'Riskakor fullkorn smaksatta fett 18%']]
Frukt o nötblandningar bars 
 [['Energibar choklad nötter Start'
  'Müslibar fullkorn berikad typ Special K Bar Red fruit']]
Frukt o bär 
 [['Havtorn' 'Aronia']]
Sportdrycker energidrycker 
 [['Sportdryck drickf' 'Energidryck typ Red Bull berikad']]
Gelatin agar agar 
 [['Agar torkad' 'Gelatinblad gelatinpulver']]
Övrigt 
 [['Samarinpulver' 'Samarin drickf']]
Blodmat 
 [['Gris blod rå' 'Nöt blod rå']]
Tuggummi 
 [['Tuggummi' 'Tuggummi sockerfritt']]
Kakaoprodukter 
 [['Kakaopulver fett 20-22%' "Kakaopulver m socker fett 2,5% typ O'boy"]]
Cider alkol

Den här klassificeringen är alltså den som Livsmedelsverket själva har gjort, och det går att ändra parametrarna här ovan och få fram mycket mer om den.

Men vad händer om man baserar en gruppering bara på näringsvärden i stället?

## Klustring av näringsvärden

Det finns över 50 näringsvärden i databasen.

Av de över 50 näringsvärdena får man välja ut ett antal som man tror kan spela roll för att ge livsmedlen en viss "profil". 

I princip skulle man kunna ha alla näringsvärden med, och man kan testa sig fram. Men på något sätt tror jag att de näringsvärden där många livsmedel har noll-värden kommer att förvirra klustringen.

De här tror jag ger en bra bild av rymden av livsmedel. 
```
[2 'Energi_kcal' 'REAL' 0 None 0]
[4 'Kolhydrater_g' 'REAL' 0 None 0]
[5 'Fett_g' 'REAL' 0 None 0]
[6 'Protein_g' 'REAL' 0 None 0]
[7 'Fibrer_g' 'REAL' 0 None 0]
[8 'Vatten_g' 'REAL' 0 None 0]
[9 'Alkohol_g' 'REAL' 0 None 0]
[10 'Aska_g' 'REAL' 0 None 0]
[42 'Vitamin_C_mg' 'REAL' 0 None 0]
[50 'Jarn_mg' 'REAL' 0 None 0]
```

In [9]:
#Här är det urval som används nu
relevant_columns = [ 2,  4,  5,  6,  7,  8,  9, 10, 42, 50]

#Här är alla möjliga dimensioner. En del försvinner eftersom de 
#har värden som saknas för vissa livsmedel
#[ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57]

Variabeln columns används för att hålla de värden som ska klustras. Vid klustringen undersöker algoritmen om man kan hitta några grupper av livsmedel som är mer lika än andra. Den slumpar ut kluster-mittpunkter och ser vilka livsmedel som ligger närmast dem (i alla dimensioner). Därefter ändrar den mittpunkterna lite, räknar om och ser om det blev bättre.

Lite om numpy: i numpy kan man använda en array som index, men det gäller att göra det till ett index för kolumnerna och inte för raderna... (Läs mer om array som index här: <http://infontology.typepad.com/infontokod/2017/11/bra-funktioner-i-numpy.html> Där står också om den här "slice"-notationen array[a:b,c:d])

In [10]:
columns = dataset[:,relevant_columns] #Tar man bort ':,' blir det alltså ett urval rader
print(columns)

[[656.3 0.0 71.0 ..., 0.3 0.0 0.3]
 [763.0 0.0 85.0 ..., 0.7 0.0 0.2]
 [884.3 0.0 100.0 ..., 0.0 0.0 0.1]
 ..., 
 [40.6 7.6 0.5 ..., 0.2 0.0 0.1]
 [372.6 72.6 3.55 ..., 0.0 0.0 11.35]
 [313.7 57.5 4.8 ..., 0.0 0.0 0.85]]


Det går förstås att välja en massa olika klustringsalgoritmer. Vi har valt k-means. Det finns också en hel del parametrar att välja. <http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html>

In [11]:
 def cluster(dataset):
    kmeans = KMeans(n_clusters=numClusters, verbose=0, n_init=100)
    kmeans.fit(dataset)

    centroids = kmeans.cluster_centers_
    clusters = kmeans.labels_
    
    #Utskriften kommer ju inte här, utan efter funktionsanropet
    if debug:
        print (centroids)
        print (clusters)

    return clusters, centroids

In [12]:
#Livsmedelsdatabasen innehåller över 100 huvudgrupper. Det hade varit bra att ha något 
#färre grupper på översta nivån. Högst 50. Eller kanske bara 10, för att komma ner i något som
#skulle kunna motsvara "frukt, grönsaker, fisk, kött, nötter" etc.
numClusters = 20

In [13]:
clusters, centroids = cluster(columns)

[[  2.29660248e+02   1.13204348e+01   1.39391925e+01   1.20080124e+01
    8.23913043e-01   5.73424845e+01   1.44546584e+00   3.11267081e+00
    1.34347826e+00   1.54440994e+00]
 [  5.16830952e+02   4.97550000e+01   3.15292857e+01   7.39452381e+00
    3.09738095e+00   6.29547619e+00  -2.77555756e-17   1.91547619e+00
    9.66666667e-01   1.91333333e+00]
 [  6.54232210e+01   9.13584270e+00   1.29812734e+00   3.10382022e+00
    1.57606742e+00   8.37294382e+01   1.71310861e-01   9.95093633e-01
    7.61722846e+00   8.93558052e-01]
 [  3.45908772e+02   6.41264912e+01   3.45692982e+00   1.02463158e+01
    6.88605263e+00   1.27913158e+01   3.88578059e-16   2.50859649e+00
    1.29912281e+00   3.25464912e+00]
 [  8.84300000e+02   0.00000000e+00   1.00000000e+02   1.77635684e-15
    4.44089210e-16  -2.13162821e-14   8.32667268e-17   4.44089210e-16
    1.77635684e-15   6.25000000e-03]
 [  7.10170968e+02   1.47935484e+00   7.84309677e+01   2.24387097e+00
    9.08387097e-01   1.55654839e+01  -2.77555

Clusters är en array som är lika lång som antalet rader i dataset:

In [14]:
print(clusters, "Antal värden:", len(clusters))

[ 5  5  4 ..., 12 19  3] Antal värden: 2088


Beroende på hur man väljer parametrarna vid klustringen kan man få liten eller stor variation i hur många som hamnar i de olika klustren.

Här är en tabell med fördelningen, och med några enkla statistiska mått.

In [15]:
unique, counts = np.unique(clusters, return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
cluster_distribution = np.array(list(zip(unique, counts)))
print(cluster_distribution)
print("Medelvärde:", np.mean(cluster_distribution.T[1]))
print("Median:", np.median(cluster_distribution.T[1]))
print("Standardavvikelse:",np.std(cluster_distribution.T[1]))

[[  0 161]
 [  1  42]
 [  2 267]
 [  3 114]
 [  4  16]
 [  5  31]
 [  6 250]
 [  7 105]
 [  8  14]
 [  9  65]
 [ 10 297]
 [ 11  61]
 [ 12 188]
 [ 13   4]
 [ 14  61]
 [ 15  87]
 [ 16 213]
 [ 17   4]
 [ 18  31]
 [ 19  77]]
Medelvärde: 104.4
Median: 71.0
Standardavvikelse: 90.5540722442


## Huvudgrupper i relation till kluster

Jag vet inte hur Livsmedelsverket har gjort när de har satt ihop sin lista av huvudgrupper. Jag tror att den bygger mycket på tradition.

När jag började titta på klustring begränsade jag beräkningarna till bara fett, protein och kolhydrater, och då hamnade mesosten i samma kluster som knäckebrödet, på grund av sin höga andel kolhydrater. Det var lite oväntat, men rymden av näringsvärden är ju helt ovetande om mänskliga måltidstraditioner eller inköpsmönster.

En av poängerna med att titta på kluster är att få en uppfattning just om vilka livsmedel som hänger ihop näringsmässigt. Om man vill skulle det sen vara möjligt att ta ett visst livsmedel, exempelvis _bläckfisk_, och fråga vilka andra livsmedel som näringsmässigt ligger nära bläckfisken. Det kanske visar sig att tofu eller drickyoghurt ligger lika nära som torsk eller blåmusslor.

Nu efter att klustringen är gjord går det att se vilka livsmedel som ingår i varje kluster, och vilka huvudgrupper de ingår i.

In [16]:
valt_kluster = 3

In [17]:
livsmedel_i_kluster = dataset[np.where(clusters==valt_kluster)][:,[0,60]]
print(livsmedel_i_kluster)

[['Mesost lätt fett 9% berik' 'Mesvaror']
 ['Kondenserad mjölk konserv konc sockrad fett ca 9%' 'Mjölk']
 ['Mjölkpulver fett 1%' 'Mjölk']
 ['Hårt bröd fullkorn råg fibrer 15,5% Wasa husman' 'Hårt bröd']
 ['Hårt bröd fullkorn råg fibrer ca 18% Ryvita mörkt' 'Hårt bröd']
 ['Hårt bröd fullkorn råg fibrer 14,5% Vika knäckebröd' 'Hårt bröd']
 ['Hårt bröd fullkorn råg fibrer 16% Crisp o Finn crisp' 'Hårt bröd']
 ['Hårt bröd fullkorn råg fibrer ca 15% Kavli flatbröd' 'Hårt bröd']
 ['Hårt bröd glutenfritt fibrer ca 7%' 'Hårt bröd']
 ['Hårt bröd råg fullkorn fibrer ca 13% ICA handlarnas knäckebröd'
  'Hårt bröd']
 ['Hårt bröd fullkorn råg kli fibrer ca 15% Siljans kraftknäcke'
  'Hårt bröd']
 ['Hårt bröd fullkorn råg fibrer ca 14% Falu råg-rut' 'Hårt bröd']
 ['Hårt bröd fullkorn råg fibrer 15,5% Wasa brungräddat' 'Hårt bröd']
 ['Hårt bröd fullkorn råg fibrer ca 16% Wasa sport' 'Hårt bröd']
 ['Hårt bröd fullkorn råg sesamfrö vetekli vetegroddar fibrer 24% Wasa plus'
  'Hårt bröd']
 ['Hårt bröd f

Eftersom det är så många livsmedel i varje kluster är det lättare att visa dem som en fördelning:

In [18]:
a=dataset[np.where(clusters==valt_kluster)][:,[0,60]]
unique, counts = np.unique(a.T[1], return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
huvudgrupp_distribution = np.array(list(zip(unique, counts)))

print(huvudgrupp_distribution)

[['' '1']
 ['Baljväxter (bönor, linser och ärter)' '8']
 ['Buljong' '1']
 ['Bullar kakor tårtor mm' '10']
 ['Efterrätter' '1']
 ['Flingor - frukostflingor' '5']
 ['Frukt o bär torkade' '1']
 ['Gelatin agar agar' '1']
 ['Godis ej choklad' '4']
 ['Hårt bröd' '19']
 ['Jäst bakpulver' '1']
 ['Matgryn' '4']
 ['Mesvaror' '1']
 ['Mjukt bröd' '1']
 ['Mjöl stärkelse kli' '26']
 ['Mjölk' '2']
 ['Pasta' '6']
 ['Potatis' '1']
 ['Ris risnudlar' '13']
 ['Socker sirap honung' '2']
 ['Sås dressing majonnäs' '1']
 ['Söta soppor kräm o efterrättssås' '2']
 ['Te' '1']
 ['Tuggummi' '1']
 ['Välling' '1']]


Om man sorterar blir det lättare att få en uppfattning av vad klustret egentligen innehåller. 

In [19]:
order=np.argsort(huvudgrupp_distribution.T[1].astype(int))
order = np.flipud(order)
#print (order)
huvudgrupp_distribution = huvudgrupp_distribution[order]
print(huvudgrupp_distribution)

[['Mjöl stärkelse kli' '26']
 ['Hårt bröd' '19']
 ['Ris risnudlar' '13']
 ['Bullar kakor tårtor mm' '10']
 ['Baljväxter (bönor, linser och ärter)' '8']
 ['Pasta' '6']
 ['Flingor - frukostflingor' '5']
 ['Matgryn' '4']
 ['Godis ej choklad' '4']
 ['Mjölk' '2']
 ['Socker sirap honung' '2']
 ['Söta soppor kräm o efterrättssås' '2']
 ['Gelatin agar agar' '1']
 ['Efterrätter' '1']
 ['Buljong' '1']
 ['Frukt o bär torkade' '1']
 ['Välling' '1']
 ['Mesvaror' '1']
 ['Jäst bakpulver' '1']
 ['Tuggummi' '1']
 ['Mjukt bröd' '1']
 ['Potatis' '1']
 ['Sås dressing majonnäs' '1']
 ['Te' '1']
 ['' '1']]


När jag körde fick jag en gång dessa som största kategorier i ett kluster:
    
```
['Korv' '17']
['Kött färskt fryst tillagat' '16']
['Pizza paj pirog färdig smörgås' '13']
['Köttprodukter kötträtter' '12']
['Fisk o skaldjursprodukter o rätter' '12']
['Starksprit' '6']
```

Jag antar att det hänger ihop med det höga energiinnehållet i spriten. Det är ju ingen som skulle föreslå att byta ut korven i middagen mot en sup, men kanske kan man tänka omvänt, att om man dricker mycket alkohol så motsvarar det rätt mycket korv.

Men även om man tycker att man har fått en stor andel av en viss huvudgrupp i sitt kluster, så kanske det ändå är bara en liten andel av hela huvudgruppen. 

Jag gjorde några beräkningar till, för att få fram hur stor del av huvudgruppen som man har prickat rätt:

In [20]:
totals = np.array([])
for item in huvudgrupp_distribution:
    a=np.where(huvudgrupp_storlek.T[0] == item [0])
    totals=np.append(totals,huvudgrupp_storlek[a[0],1])
samlad_array = np.concatenate((huvudgrupp_distribution.T,[totals]),axis=0).T
part = lambda x: np.divide(x[1].astype(int),x[2].astype(int))
andel = part(samlad_array.T)
samlad_array = np.vstack((samlad_array.T,andel.T))
print ("Namn,", "Antal i kluster,", "Totalt i huvudgrupp,", "Andel av huvudgrupp")
print (samlad_array.T)    

Namn, Antal i kluster, Totalt i huvudgrupp, Andel av huvudgrupp
[['Mjöl stärkelse kli' '26' '29' '0.896551724137931']
 ['Hårt bröd' '19' '28' '0.6785714285714286']
 ['Ris risnudlar' '13' '25' '0.52']
 ['Bullar kakor tårtor mm' '10' '86' '0.11627906976744186']
 ['Baljväxter (bönor, linser och ärter)' '8' '51' '0.1568627450980392']
 ['Pasta' '6' '21' '0.2857142857142857']
 ['Flingor - frukostflingor' '5' '28' '0.17857142857142858']
 ['Matgryn' '4' '14' '0.2857142857142857']
 ['Godis ej choklad' '4' '11' '0.36363636363636365']
 ['Mjölk' '2' '11' '0.18181818181818182']
 ['Socker sirap honung' '2' '6' '0.3333333333333333']
 ['Söta soppor kräm o efterrättssås' '2' '26' '0.07692307692307693']
 ['Gelatin agar agar' '1' '2' '0.5']
 ['Efterrätter' '1' '31' '0.03225806451612903']
 ['Buljong' '1' '18' '0.05555555555555555']
 ['Frukt o bär torkade' '1' '14' '0.07142857142857142']
 ['Välling' '1' '12' '0.08333333333333333']
 ['Mesvaror' '1' '6' '0.16666666666666666']
 ['Jäst bakpulver' '1' '3' '0.33

Det här är kanske så långt vi kommer i det här resonemanget. En början till att jämföra Livsmedelsverkets klassificering med den klassificering man får om man _bara_ ser till näringsvärden i olika kombinationer.

En av de saker som drev mig till detta var de småkategorier som fanns bland Huvudgrupperna. Det skulle vara så bra om man kunde stoppa in dem i större kategorier. Men kanske går det inte. I alla fall inte baserat endast på näringsinnehåll.

Det här tillvägagångssättet går förstås att effektivisera extremt mycket. Den här essän är bara skriven av pedagogiska skäl (dels för att visa potentialen i Jupyter, och dels för att visa upp rymden av livsmedelsdata).

Man skulle till nästa essä kunna hålla fast vid möjligheten att det finns en korrespondens mellan huvudgrupper och näringsvärden, och kunna se vilka kombinationer av näringsvärden (och antal kluster) som ger best match mot huvudgrupperna. Det kräver massor av beräkningar! För varje antal kluster, välj alla kombinationer av näringsvärden: först enskilda näringsvärden, sen grupper av två, grupper av tre och så vidare. Och så måste man förstås hitta ett bra mått. Det måste man kanske också pröva sig fram till... Undrar om det finns ett matematiskt sätt att hitta det också, eller om det bara är möjligt att köra det så här med numeriska metoder...